In [18]:
#pip install scikit-multilearn
#algoritmos adaptados
# Knn e Hierarquical ARAM NN
from skmultilearn.adapt import MLkNN, MLARAM
#transformação de problemas
from skmultilearn.problem_transform import BinaryRelevance, ClassifierChain, LabelPowerset
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import hamming_loss
import pandas as pd

In [19]:
musica = pd.read_csv('Musica.csv', sep=",")
musica.shape

(592, 77)

In [20]:
musica.head()

,amazed-suprised,happy-pleased,relaxing-clam,quiet-still,sad-lonely,angry-aggresive,Mean_Acc1298_Mean_Mem40_Centroid,Mean_Acc1298_Mean_Mem40_Rolloff,Mean_Acc1298_Mean_Mem40_Flux,Mean_Acc1298_Mean_Mem40_MFCC_0,...,Std_Acc1298_Std_Mem40_MFCC_10,Std_Acc1298_Std_Mem40_MFCC_11,Std_Acc1298_Std_Mem40_MFCC_12,BH_LowPeakAmp,BH_LowPeakBPM,BH_HighPeakAmp,BH_HighPeakBPM,BHSUM1,BHSUM2,BHSUM3
0,0,1,1,0,0,0,0.132498,0.077848,0.229227,0.602629,...,0.197026,0.196244,0.164323,0.030017,0.253968,0.008473,0.240602,0.136735,0.058442,0.107594
1,1,0,0,0,0,1,0.384281,0.355249,0.167190,0.853089,...,0.093526,0.085649,0.025101,0.182955,0.285714,0.156764,0.270677,0.191377,0.153728,0.197951
2,0,1,0,0,0,1,0.541782,0.356491,0.152246,0.791142,...,0.198082,0.108067,0.140574,0.099303,0.142857,0.000000,0.593985,0.105114,0.025555,0.122965
3,0,0,1,0,0,0,0.174288,0.243935,0.254326,0.438987,...,0.235793,0.220195,0.235834,0.024988,0.222222,0.117169,0.210526,0.057288,0.134575,0.091509
4,0,0,0,1,0,0,0.347436,0.155448,0.100047,0.126026,...,0.715683,0.573359,0.412368,0.016398,0.761905,0.081703,0.721805,0.108737,0.172882,0.189934


In [21]:
#separação da classe e previsores
classe = musica.iloc[:,0:6].values
previsores = musica.iloc[:,7:78].values
classe

array([[0, 1, 1, 0, 0, 0],
       [1, 0, 0, 0, 0, 1],
       [0, 1, 0, 0, 0, 1],
       ...,
       [0, 0, 1, 1, 1, 0],
       [0, 1, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 0]], dtype=int64)

In [22]:
#divisão de treino e teste
X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(previsores,classe,test_size = 0.3,
                                                                  random_state = 0)

In [23]:
#adaptado
#vizinho mais próximo para ML
vmp = MLkNN(k=3) 
vmp.fit(X_treinamento, y_treinamento) 

MLkNN(ignore_first_neighbours=0, k=3, s=1.0)

In [24]:
#previsão com dados de teste
previsto = vmp.predict(X_teste) 
#hamming para avaliar preformance  
print(hamming_loss(y_teste, previsto)) 

0.23782771535580524


In [25]:
#segundo classificador adaptado, Hierarquical ARAM NN
ann = MLARAM()
ann.fit(X_treinamento, y_treinamento) 

MLARAM(neurons=[<skmultilearn.adapt.mlaram.Neuron object at 0x0000020683642848>,
                <skmultilearn.adapt.mlaram.Ne...
                <skmultilearn.adapt.mlaram.Neuron object at 0x0000020683643E48>, ...],
       threshold=0.02, vigilance=0.9)

In [26]:
#previsão com dados de teste
previsto = ann.predict(X_teste) 
#hamming para avaliar preformance  
print(hamming_loss(y_teste, previsto)) 

0.24812734082397003


In [27]:
#transformação de problema
#binary relevance
binary = BinaryRelevance(classifier = SVC())
binary.fit(X_treinamento, y_treinamento)
previsao = binary.predict(X_teste)
print(hamming_loss(y_teste, previsao))

0.199438202247191


In [28]:
#transformação de problema
#ClassifierChain
chain = ClassifierChain(classifier = SVC())
chain.fit(X_treinamento, y_treinamento)
previsoes = chain.predict(X_teste)
print(hamming_loss(y_teste,previsoes))

0.2340823970037453


In [29]:
#transformação de problema
#labelpowerset
label = LabelPowerset(classifier = SVC())
label.fit(X_treinamento, y_treinamento)
previsoes = label.predict(X_teste)
print(hamming_loss(y_teste,previsoes))

0.2209737827715356
